# Spark SQL Tutorial
- Author: Akira Takihara Wang (https://github.com/akiratwang)
- Tutorial Up-to-Date as of: April 2021  
- Usage: For MAST30034 students only  


In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

# Start the spark context
sc = SparkContext.getOrCreate(conf=swan_spark_conf)

# create a spark session (which will run spark jobs)
spark = SparkSession.builder.getOrCreate()

# apply settings to session
spark.conf.set('spark.sql.repl.eagerEval.enabled', True)
spark.conf.set('spark.sql.execution.arrow.pyspark.enabled', True)

NameError: name 'swan_spark_conf' is not defined

Previously, we saw how to create a `schema` using PySpark code. If you are more familiar with SQL, then you may want to use a Data Definition Language (DDL) string to create a schema. 

Syntax:
```python
ddl = """
`COLUMN 1` DATATYPE1, `COLUMN 2` DATATYPE2, ...
"""
```

Although it may make sense to pass through the datetimes as type `TIMESTAMP`, the format may be inconsistent. As an example, we will assume the worst case and "manually" fix it using a function for your benefit.

In [ ]:
# create a DDL
schema = """
`VendorID` INT, `tpep_pickup_datetime` STRING, `tpep_dropoff_datetime` STRING,
`passenger_count` INT, `trip_distance` DOUBLE, `pickup_longitude` DOUBLE, `pickup_latitude` DOUBLE,
`RateCodeID` INT, `store_and_fwd_flag` STRING, `dropoff_longitude` DOUBLE, `dropoff_latitude` DOUBLE,
`payment_type` INT, `fare_amount` DOUBLE, `extra` DOUBLE, `mta_tax` DOUBLE, `tip_amount` DOUBLE,
`tolls_amount` DOUBLE, `improvement_surcharge` DOUBLE, `total_amount` DOUBLE
"""

Like PySpark, you can simply pass it through the `schema` parameter.

In [ ]:
sdf = spark.read.csv('../data/sample.csv', header=True, schema=schema)
sdf.printSchema()

- As you can see, this achieves the same result as using `StructType()`, but may be easier or more difficult depending on the number of columns you have. 
- My personal preference would be using `StructType()` for this dataset as you can use generator functions to simplify the allocation of dtypes.
- However, depending on your role or project scope, you may already have an existing DDL you can use!

In [ ]:
sdf.limit(5)

Now let's get our datetimes into `TIMESTAMP` formats (since our current dataset does not have the correct format).

In [ ]:
import pyspark.sql.functions as F

from pyspark.sql.functions import col
from pyspark.sql.types import *

In [ ]:
# create UDF
from datetime import datetime

@F.udf("timestamp")
def format_dtime(dtime):
    date, time = dtime.split()
    # map the iterable to integer
    d, m, y = map(int, date.split('/'))
    # year is abbreviated so we need to add 20 in front
    y = int(f"20{y}")
    h, mins = map(int, time.split(':'))
    return datetime(y, m, d, h, mins)

In [ ]:
sdf.withColumn("tpep_pickup_datetime", format_dtime('tpep_pickup_datetime')) \
    .withColumn("tpep_dropoff_datetime", format_dtime('tpep_dropoff_datetime')) \
    .limit(5)

- Conversion looks good to me, so let's keep it.
- Remember, Spark is immutable, so we will need to overwrite the `sdf` variable.

In [ ]:
sdf = sdf.withColumn("tpep_pickup_datetime", format_dtime('tpep_pickup_datetime')) \
    .withColumn("tpep_dropoff_datetime", format_dtime('tpep_dropoff_datetime'))

In [ ]:
sdf.limit(5)

## Creating a SQL Table with an existing Spark DataFrame
The easiest method is to use `sdf.createOrReplaceTempView(TABLE_NAME)`


In [ ]:
sdf.createOrReplaceTempView("taxi_data")

Select all columns from our table, where:
- the Vendor is `VeriFone Inc.`;
- we have at least 1 passenger;
- and a trip distance greater than 1 mile.

In [ ]:
sql_query = """
SELECT * 
FROM taxi_data
WHERE VendorID = 2
    AND passenger_count >= 1
    AND trip_distance >= 1
LIMIT 5;
"""

spark.sql(sql_query)

Below is the alternative query using PySpark. As you can see, it becomes _less interpretable_. Using Spark SQL ensures that the query is consistent and can also be run directly on the database if need be, allowing for a much more consistent way of testing queries.

In [ ]:
sdf.select(sdf.columns) \
    .filter((col('VendorID') == 2) & (col('passenger_count') >= 1) & (col('trip_distance') >= 1)) \
    .limit(5)

In [ ]:
spark.catalog.listTables()

## Creating SQL views directly from files
- If you don't have a Spark DataFrame, you can still read it in directly using Spark SQL.

Syntax:
```python
q = """
CREATE OR REPLACE TEMPORARY VIEW PARQUET_NAME
USING parquet
OPTIONS (path PARQUET_FPATH)
"""
```

- `CREATE OR REPLACE TEMPORARY VIEW TABLE_NAME` is the same as `sdf.createOrReplaceTempView(TABLE_NAME)`.
- `USING` denotes the file type (i.e `csv`).
- `OPTIONS` indicates the file path we wish to read from.

In [ ]:
import os

sql_query = f"""
CREATE OR REPLACE TEMPORARY VIEW aggregation_parquet
USING parquet
OPTIONS (path 
    "{'/'.join(os.getcwd().split('/')[:-1])}/data/aggregated_results.parquet/")
"""

spark.sql(sql_query)

In [ ]:
spark.sql("SELECT * FROM aggregation_parquet")